In [1]:
!pip install folium

In [2]:
import pandas as pd
import folium
from folium import plugins
import numpy as np

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/AI활용 소프트웨어 개발/11. 데이터 분석/data/banapresso.csv')

print("=" * 60)
print(" 바나프레소 매장 지도 시각화")
print("=" * 60)
print(f" 총 매장 수: {len(df)}개")
print(f" 서울 매장: {len(df[df['addr'].str.contains('서울', na=False)])}개")
print("=" * 60)

 바나프레소 매장 지도 시각화
 총 매장 수: 110개
 서울 매장: 85개


In [ ]:
print("\n 데이터 미리보기:")
print(df.head())


 데이터 미리보기:
       store                             addr        lat         lng
0  가산디지털단지역점                 서울시 금천구 가산동 60-3  37.453829  126.897298
1     가산안양천점    서울 금천구 가산 디지털2로 127-143, 101호  37.458424  126.889594
2    가산어반워크점     서울시 금천구 가산디지털2로 135, 1동 142호  37.460721  126.903415
3  가산우림라이온스점  서울특별시 금천구 가산디지털1로 168, B동 B131호  37.466110  126.899920
4  가산파트너스타워점     서울특별시 금천구 가산디지털1로 83, 103호-1  37.460596  126.891223


In [ ]:
def get_region_color(address):
    """주소에 따른 마커 색상 반환"""
    if '강남구' in address:
        return 'red'
    elif '서초구' in address:
        return 'blue'
    elif '송파구' in address:
        return 'green'
    elif '강동구' in address:
        return 'purple'
    elif '구로구' in address:
        return 'orange'
    elif '금천구' in address:
        return 'darkred'
    elif '마포구' in address:
        return 'lightred'
    elif '영등포구' in address:
        return 'beige'
    elif '용산구' in address:
        return 'darkblue'
    elif '중구' in address:
        return 'darkgreen'
    elif '성동구' in address:
        return 'cadetblue'
    elif '광진구' in address:
        return 'darkpurple'
    elif '동대문구' in address:
        return 'pink'
    elif '성북구' in address:
        return 'gray'
    elif '관악구' in address:
        return 'black'
    elif '동작구' in address:
        return 'lightblue'
    elif '강북구' in address:
        return 'lightgreen'
    elif '서울' in address:
        return 'blue'
    elif '경기' in address:
        return 'lightgray'
    elif '인천' in address:
        return 'white'
    else:
        return 'gray'

def get_region_icon(address):
    """주소에 따른 아이콘 반환"""
    if '서울' in address:
        return 'star'
    elif '경기' in address:
        return 'home'
    elif '인천' in address:
        return 'ship'
    elif '부산' in address:
        return 'plane'
    else:
        return 'info-sign'

In [ ]:
center_lat = df['lat'].mean()
center_lng = df['lng'].mean()

m = folium.Map(
    location=[center_lat, center_lng],
    zoom_start=11,
    tiles='OpenStreetMap'
)

In [ ]:
print("\n 지도에 매장 마커 추가 중...")

for idx, row in df.iterrows():
    # 팝업 정보 생성
    popup_html = f"""
    <div style="font-family: Arial; width: 200px;">
        <h4 style="color: #d4691b; margin-bottom: 10px;">☕ {row['store']}</h4>
        <p style="margin: 5px 0;"><b> 주소:</b><br>{row['addr']}</p>
        <p style="margin: 5px 0;"><b> 좌표:</b><br>위도: {row['lat']:.6f}<br>경도: {row['lng']:.6f}</p>
    </div>
    """

    # 마커 추가
    folium.Marker(
        location=[row['lat'], row['lng']],
        popup=folium.Popup(popup_html, max_width=300),
        tooltip=f"☕ {row['store']}",
        icon=folium.Icon(
            color=get_region_color(row['addr']),
            icon=get_region_icon(row['addr']),
            prefix='fa'
        )
    ).add_to(m)


 지도에 매장 마커 추가 중...


In [ ]:
marker_cluster = plugins.MarkerCluster().add_to(m)

for idx, row in df.iterrows():
    folium.Marker(
        location=[row['lat'], row['lng']],
        popup=f"☕ {row['store']}<br>📍 {row['addr']}",
        tooltip=row['store']
    ).add_to(marker_cluster)

In [ ]:
legend_html = """
<div style="position: fixed;
     bottom: 50px; left: 50px; width: 200px; height: 300px;
     background-color: white; border:2px solid grey; z-index:9999;
     font-size:14px; padding: 10px">

<h4 style="color: #d4691b; margin-bottom: 10px;"> 지역별 색상</h4>

<p><i style="color:red">●</i> 강남구 (27개)</p>
<p><i style="color:blue">●</i> 서초구 (16개)</p>
<p><i style="color:green">●</i> 송파구</p>
<p><i style="color:purple">●</i> 강동구</p>
<p><i style="color:orange">●</i> 구로구 (5개)</p>
<p><i style="color:darkred">●</i> 금천구 (5개)</p>
<p><i style="color:lightred">●</i> 마포구</p>
<p><i style="color:gray">●</i> 기타 지역</p>

<hr>
<p style="font-size:12px; color:gray;">
총 110개 매장<br>
서울: 85개 (77%)
</p>

</div>
"""

m.get_root().html.add_child(folium.Element(legend_html))

In [ ]:
heat_data = [[row['lat'], row['lng']] for idx, row in df.iterrows()]
plugins.HeatMap(heat_data, radius=15, blur=10, max_zoom=1).add_to(m)

In [ ]:
print("\n 지역별 매장 분포:")
seoul_regions = ['강남구', '서초구', '송파구', '강동구', '구로구', '금천구', '마포구', '영등포구']

for region in seoul_regions:
    count = len(df[df['addr'].str.contains(region, na=False)])
    if count > 0:
        print(f"   {region}: {count}개")

print(f"\n기타 지역: {len(df) - len(df[df['addr'].str.contains('|'.join(seoul_regions), na=False)])}개")


 지역별 매장 분포:
   강남구: 27개
   서초구: 16개
   송파구: 2개
   강동구: 3개
   구로구: 5개
   금천구: 5개
   마포구: 2개
   영등포구: 3개

기타 지역: 47개


In [ ]:
print("\n 지도 저장 중...")

m.save('banapresso_map.html')
print(" 지도가 'banapresso_map.html'로 저장되었습니다!")

print("\n 지도를 표시합니다:")
m


 지도 저장 중...
 지도가 'banapresso_map.html'로 저장되었습니다!

 지도를 표시합니다:


In [ ]:
def analyze_districts():
    districts = {}
    for addr in df['addr']:
        for district in ['강남구', '서초구', '송파구', '강동구', '구로구', '금천구', '마포구', '영등포구', '용산구', '중구']:
            if district in addr:
                districts[district] = districts.get(district, 0) + 1
                break

    print("\n 매장 수 TOP 5:")
    sorted_districts = sorted(districts.items(), key=lambda x: x[1], reverse=True)
    for i, (district, count) in enumerate(sorted_districts[:5], 1):
        print(f"   {i}. {district}: {count}개")

analyze_districts()

print("\n" + "=" * 60)
print(" 바나프레소 매장 지도 시각화 완료!")
print("   - 빨간색 마커가 가장 많은 강남구")
print("   - 파란색 마커가 두 번째로 많은 서초구")
print("   - 클러스터 기능으로 확대/축소 가능")
print("   - 히트맵으로 매장 밀도 확인 가능")
print("=" * 60)


 매장 수 TOP 5:
   1. 강남구: 27개
   2. 서초구: 16개
   3. 중구: 6개
   4. 금천구: 5개
   5. 구로구: 5개

 바나프레소 매장 지도 시각화 완료!
   - 빨간색 마커가 가장 많은 강남구
   - 파란색 마커가 두 번째로 많은 서초구
   - 클러스터 기능으로 확대/축소 가능
   - 히트맵으로 매장 밀도 확인 가능
